# Анализатор текстовых пресс-релизов

Задача: На основании исторических пресс-релизов кредитных рейтинговых агентств участникам хакатона необходимо построить интерпретируемую ML-модель, устанавливающую взаимосвязь между текстом пресс-релиза и присвоенным кредитным рейтингом по национальной рейтинговой шкале Российской Федерации для организации с учетом методологических особенностей оценки рейтинга. ML-модель должна не просто устанавливать соответствие текста пресс-релиза кредитному рейтингу, но также и выделять ключевые конструкции в тексте, соответствующие присвоенному кредитному рейтингу.



## План

Исходя из поставленной зажачи мы сформировали план
1. Загрузить данные
2. Оценить данные, если нужно, почистить, лематизировать
3. Использовать для токенизации инструмент TfidfVectorizer
4. Построить на основе полученных векторов разные модели предсказания кредитного рейтинга
5. Использовать трансформер BERT
6. Построить на основе полученных векторов разные модели предсказания кредитного рейтинга
7. Оценить результаты и сделать выводы

In [3]:
import numpy as np
import pandas as pd
import re
import torch
from tqdm import notebook
from sklearn.model_selection import train_test_split, GridSearchCV
from pymystem3 import Mystem
m = Mystem()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import torch
import transformers as ppb
import lightgbm as lgb
from pytorch_transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from nltk.corpus import wordnet
import wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk import word_tokenize
import requests
from tqdm import tqdm
tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from nltk.probability import FreqDist
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

from lightgbm import LGBMClassifier

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [25]:
data = pd.read_excel('CRA_train_1200.xlsx')

In [36]:
data.head(5)


,Id,text,category,rating
0,1,Повышение кредитного рейтинга Акционерного об...,A,A
1,2,«Эксперт РА» подтвердил кредитный рейтинг комп...,BB,BB
2,3,"НКР повысило кредитный рейтинг ООО ""ОТЭКО-Порт...",A,A
3,4,«Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...,AAA,AAA
4,5,29 марта 2023 г. Ведущий рейтинговый аналитик ...,BBB,BBB


In [39]:
data = data.rename(columns={
    'pr_txt': 'text',
    'Категория': 'category',
    'Уровень рейтинга': 'rating'
})


In [11]:
data['rating'].unique()

array(['A', 'BB', 'AAA', 'BBB', 'AA+', 'BB+', 'BB-', 'A-', 'A+', 'B',
       'AA-', 'BBB+', 'BBB-', 'B-', 'AA', 'B+', 'C'], dtype=object)

In [12]:
category_mapping = {
    'C': 0, 
    'B': 1, 
    'BB': 2, 
    'BBB': 3, 
    'A': 4, 
    'AA': 5, 
    'AAA': 6
}

data['category_num'] = data['category'].replace(category_mapping)

rating_mapping = {
    'C': 0, 
    'B-': 1, 
    'B': 2, 
    'B+': 3, 
    'BB-': 4, 
    'BB': 5, 
    'BB+': 6, 
    'BBB-': 7, 
    'BBB': 8, 
    'BBB+': 9, 
    'A-': 10, 
    'A': 11, 
    'A+': 12, 
    'AA-': 13, 
    'AA': 14, 
    'AA+': 15, 
    'AAA': 16
}

data['rating_num'] = data['rating'].replace(rating_mapping)


In [40]:
def cleaning(text):
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Zа-яА-Я ]+", "", text).strip()
    text = text.lower()
    return text

data['text'] = data['text'].apply(cleaning)

In [41]:
from pymystem3 import Mystem
from nltk.corpus import stopwords
import nltk
import pandas as pd
from tqdm import tqdm

nltk.download('stopwords')

m = Mystem()

# Загрузка русских стоп-слов
russian_stopwords = stopwords.words("russian")



def lemmatize_text(corpus):
    corpus_new = []
    for sentence in tqdm(corpus):
        lemmatized_sentence = m.lemmatize(sentence)
        cleaned_sentence = ' '.join([word for word in lemmatized_sentence if word.strip() and word not in russian_stopwords])
        corpus_new.append(cleaned_sentence)
    return corpus_new



data['text_lemmatized'] = lemmatize_text(data['text'])

print(data.head())


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Денис\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 1200/1200 [18:04<00:00,  1.11it/s]

   Id                                               text category rating  \
0   1  повышение кредитного рейтинга  акционерного об...        A      A   
1   2  эксперт ра подтвердил кредитный рейтинг компан...       BB     BB   
2   3  нкр повысило кредитный рейтинг ооо отэкопортсе...        A      A   
3   4  эксперт ра присвоил кредитный рейтинг пао фоса...      AAA    AAA   
4   5  марта  г ведущий рейтинговый аналитик юрова ал...      BBB    BBB   

                                     text_lemmatized  
0  повышение кредитный рейтинг акционерный общест...  
1  эксперт ра подтверждать кредитный рейтинг комп...  
2  нкр повышать кредитный рейтинг ооо отэкопортсе...  
3  эксперт ра присваивать кредитный рейтинг пао ф...  
4  марта г ведущий рейтинговый аналитик юров алла...  


In [6]:
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from nltk.corpus import stopwords as nltk_stopwords
import pandas as pd

# Загрузите данные
data = pd.read_csv('data.csv')

nltk_stopwords = list(nltk_stopwords.words('russian'))
extra_stopwords = [
    'далее', 'также', 'что', 'в', 'с', 'и', 'на', 'по', 'а', 'за', '—', 'как', 
    'у', 'до', 'средней', 'очень', 'один', 'это', 'средняя', '«', '»', '(', ')', 
    '№', '—', 'году', 'одной', 'посредством', 'ранее', 'большого', 'которая', 
    'который', 'этом', 'является', 'один', 'другой', 'доли', 'доля', 'их', 
    'которого', 'его', 'средний', 'средние', 'уровень', 'производителей', 
    'высокие', 'низкой', 'средней', 'продукции', 'компании', 'компания', 
    'продукция', 'оценка', 'оценки', 'оценку', 'показатели', 'показатель', 
    'факторы', 'фактор', 'уровня', 'профиля', 'бизнес', 'бизнеса', 'уровень', 
    'уровня', 'рейтинг', 'рейтинга', 'рейтингу', 'производства', 'производство'
]
nltk_stopwords.extend(extra_stopwords)
count_tf_idf = TfidfVectorizer(stop_words=nltk_stopwords)

def train_model(target_col):
    # Извлекаем признаки и целевую переменную
    X = data['text']
    y = data[target_col].values

    # Разделите данные на тренировочные и тестовые наборы
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

    # Примените TF-IDF векторизатор к вашим данным
    tfidf_train = count_tf_idf.fit_transform(X_train)
    tfidf_test = count_tf_idf.transform(X_test)

    # Определите конвейер и сетку параметров для поиска по сетке
    pipe = Pipeline([
        ('model', LogisticRegression(random_state=1, solver='liblinear', max_iter=200))
    ])

    param_grid = [
    {
        'model': [LogisticRegression(random_state=42, solver='liblinear')],
        'model__C': list(range(1, 15, 3)),
        'model__penalty': ['l1', 'l2']
    },
    {
        'model': [RandomForestClassifier(random_state=42)],
        'model__n_estimators': [100, 200, 300],
        'model__max_depth': [None, 10, 20, 30]
    },
    {
        'model': [SVC(random_state=42)],
        'model__C': [0.1, 1, 10],
        'model__kernel': ['linear', 'rbf']
    },
    {
        'model': [MultinomialNB()],
        'model__alpha': [0.1, 1.0, 10.0]
    }
    ]

        # Проведите поиск по сетке, чтобы найти наилучшие параметры
    grid = GridSearchCV(pipe, param_grid=param_grid, scoring=make_scorer(f1_score, average='weighted'), cv=5, verbose=True, n_jobs=-1)
    best_grid = grid.fit(tfidf_train, y_train)
    
    # Выведите наилучшие параметры и оценку
    print(f"Best parameters for {target_col} are:", grid.best_params_)
    print(f"Best cross-validation score for {target_col} is:", grid.best_score_)

    # Проверка на тестовой выборке
    tfidf_test = count_tf_idf.transform(X_test)
    test_predictions = best_grid.predict(tfidf_test)
    test_score = f1_score(y_test, test_predictions, average='weighted')
    print(f"Test F1 score for {target_col} is:", test_score)
    
    return grid.best_score_, test_score

# Получите лучшие оценки для каждой целевой переменной
rating_cv_f1, rating_test_f1 = train_model('rating')
category_cv_f1, category_test_f1 = train_model('category')

# Рассчитайте взвешенный итоговый результат
final_cv_score = (rating_cv_f1 * 0.65) + (category_cv_f1 * 0.35)
final_test_score = (rating_test_f1 * 0.65) + (category_test_f1 * 0.35)

print(f"Final CV F1 score for 'rating': {rating_cv_f1}")
print(f"Final CV F1 score for 'category': {category_cv_f1}")
print(f"Overall final CV score: {final_cv_score}")

print(f"Final TEST F1 score for 'rating': {rating_test_f1}")
print(f"Final TEST F1 score for 'category': {category_test_f1}")
print(f"Overall final TEST score: {final_test_score}")


Fitting 5 folds for each of 31 candidates, totalling 155 fits
Best parameters for rating are: {'model': SVC(C=10, kernel='linear', random_state=42), 'model__C': 10, 'model__kernel': 'linear'}
Best cross-validation score for rating is: 0.6108157606282278
Test F1 score for rating is: 0.6761702475812629
Fitting 5 folds for each of 31 candidates, totalling 155 fits
Best parameters for category are: {'model': LogisticRegression(C=13, penalty='l1', random_state=42, solver='liblinear'), 'model__C': 13, 'model__penalty': 'l1'}
Best cross-validation score for category is: 0.785018814950238
Test F1 score for category is: 0.7969036640890367
Final CV F1 score for 'rating': 0.6108157606282278
Final CV F1 score for 'category': 0.785018814950238
Overall final CV score: 0.6717868296409314
Final TEST F1 score for 'rating': 0.6761702475812629
Final TEST F1 score for 'category': 0.7969036640890367
Overall final TEST score: 0.7184269433589837


In [13]:
from sklearn.metrics import mean_squared_error, r2_score, f1_score, make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import pandas as pd

# Загрузите данные и мэппинги
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk_stopwords = stopwords.words('russian')

count_tf_idf = TfidfVectorizer(stop_words=nltk_stopwords)

def train_model(target_col):
    # Извлекаем признаки и целевую переменную
    X = data['text']
    y = data[target_col].values

    # Разделите данные на тренировочные и тестовые наборы
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

    # Примените TF-IDF векторизатор к вашим данным
    tfidf_train = count_tf_idf.fit_transform(X_train)
    tfidf_test = count_tf_idf.transform(X_test)

    # Определите конвейер и сетку параметров для поиска по сетке
    pipe = Pipeline([
        ('model', LinearRegression())
    ])

    param_grid = [
    {
        'model': [LinearRegression()],
    },
    {
        'model': [RandomForestRegressor(random_state=42)],
        'model__n_estimators': [100, 200, 300],
        'model__max_depth': [None, 10, 20, 30]
    },
    {
        'model': [SVR()],
        'model__C': [0.1, 1, 10],
        'model__kernel': ['linear', 'rbf']
    },
    ]

    # Проведите поиск по сетке, чтобы найти наилучшие параметры
    grid = GridSearchCV(pipe, param_grid=param_grid, scoring='r2', cv=5, verbose=True, n_jobs=-1)
    best_grid = grid.fit(tfidf_train, y_train)
    
    # Выведите наилучшие параметры и оценку
    print(f"Best parameters for {target_col} are:", grid.best_params_)
    print(f"Best cross-validation score for {target_col} is:", grid.best_score_)

    # Проверка на тестовой выборке
    test_predictions = best_grid.predict(tfidf_test)

    # Восстановим буквенные рейтинги
    inv_category_mapping = {v: k for k, v in category_mapping.items()}
    inv_rating_mapping = {v: k for k, v in rating_mapping.items()}

    if target_col == 'category_num':
        test_predictions = [inv_category_mapping[round(pred)] for pred in test_predictions]
        y_test = [inv_category_mapping[val] for val in y_test]
    elif target_col == 'rating_num':
        test_predictions = [inv_rating_mapping[round(pred)] for pred in test_predictions]
        y_test = [inv_rating_mapping[val] for val in y_test]

    # Вычислим F1-оценку
    test_score = f1_score(y_test, test_predictions, average='weighted')
    print(f"Test F1 score for {target_col} is:", test_score)
    
    return grid.best_score_, test_score

# Получите лучшие оценки для каждой целевой переменной
rating_cv_r2, rating_test_f1 = train_model('rating_num')
category_cv_r2, category_test_f1 = train_model('category_num')

# Выведите итоговые результаты
print(f"Final CV R2 score for 'rating': {rating_cv_r2}")
print(f"Final CV R2 score for 'category': {category_cv_r2}")

print(f"Final TEST F1 score for 'rating': {rating_test_f1}")
print(f"Final TEST F1 score for 'category': {category_test_f1}")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KarimovDO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Fitting 5 folds for each of 19 candidates, totalling 95 fits


In [5]:
data = pd.read_csv('data.csv')

Рассмотрим несколько стратегий для создания новых признаков:

Использование Финансовых Терминов
Частота финансовых терминов: Создайте признаки, которые представляют собой количество упоминаний ключевых финансовых терминов, таких как "долг", "прибыль", "риск" и т. д., в каждом тексте.

Sentiment Analysis на финансовых терминах: Проведите анализ тональности сосредоточиваясь на предложениях, в которых упоминаются финансовые термины, чтобы оценить, является ли контекст положительным, отрицательным или нейтральным.

Использование Метрических Данных
Числовые метрики: Создайте признаки, представляющие количество числовых упоминаний в тексте, или даже более сложные метрики, такие как среднее или медианное значение упоминаемых чисел.

Упоминание финансовых показателей: Определите, упоминаются ли специфические финансовые показатели (например, EBITDA, P/E ratio) и создайте бинарные признаки на основе их наличия или отсутствия.

Прочие Стратегии
Кластеризация текстов: Попробуйте использовать алгоритмы кластеризации для группировки текстов по схожести тем, и используйте метки кластеров как признаки.

Создание тематических дикционариев: Создайте словари с положительными и отрицательными финансовыми терминами и используйте их для создания признаков на основе числа положительных и отрицательных слов в тексте.

Длина текста и структура предложения: Включите признаки, связанные с длиной текста или структурой предложения (например, средняя длина предложения).

In [4]:
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import nltk
4

# Убедитесь, что nltk корпусы загружены
nltk.download('punkt')

# Получаем уникальные значения рейтинга
unique_ratings = data['rating'].unique()

for rating in unique_ratings:
    text_tokens = []

    # Получаем тексты для текущего рейтинга и проводим токенизацию
    for i in tqdm(data[data['rating'] == rating].text_lemmatized):
        token_vr = word_tokenize(i)
        for j in token_vr:
            text_tokens.append(j)
    
    # Создаем объект nltk.Text
    text = nltk.Text(text_tokens)
    
    # Создаем объект FreqDist и строим график
    fdist = FreqDist(text)
    fdist.plot(30, cumulative=False)
    
    # Создаем облако слов
    wordcloud = WordCloud(
        background_color='white',
        max_font_size=200,
        width=1000, height=800,
        random_state=42,
    ).generate(" ".join(text))

    # Строим график облака слов
    fig = plt.figure(figsize=(12, 14))
    plt.imshow(wordcloud)
    plt.title(f"Rating: {rating}", fontsize=45)
    plt.axis('off')
    plt.show()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KarimovDO\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


NameError: name 'data' is not defined

In [7]:
import pandas as pd

# Список признаков и соответствующие им ключевые слова для поиска в тексте
features_keywords = {
    "mlrd": ["млрд"],
    "mln": ["млн"],
    "rating_assessment": ["rating"],
    "regulatory_requirements": ["регуляторный"],
    "disclosure": ["раскрытие"],
    "credit_rating": ["кредитный"],
    "joint_stock_company": ["общество"],
    "bond_issue": ["облигаций"],
    "RUAQ": ["RUAQ"],
    "national_scale": ["шкала"],
    "non_financial_company": ["компания"],
    "agency": ["агентство"],
    "rating_activity": ["деятельность"],
    "financial_instrument": ["финансовый"],
    "AKR_credit_rating": ["AKR"],
    "credit_rating_forecast": ["прогноз"],
    "press_release": ["пресс-релиз"],
    "open_source_information": ["информация"],
    "AKR_database": ["база данных"],
    "reporting": ["отчетность"],
    "additional_services": ["услуги"],
    "conflict_of_interest": ["интересов"],
    "connection_category": ["категория"],
    "level": ["уровень"],
    "senior_unsecured_debt": ["долг"],
    "emission": ["эмиссии"],
    }

for feature, keywords in features_keywords.items():
    data[feature] = data["text_lemmatized"].apply(lambda x: any(keyword in x for keyword in keywords))
print(data.head())


   Unnamed: 0  Id                                               text category  \
0           0   1  повышение кредитного рейтинга  акционерного об...        A   
1           1   2  эксперт ра подтвердил кредитный рейтинг компан...       BB   
2           2   3  нкр повысило кредитный рейтинг ооо отэкопортсе...        A   
3           3   4  эксперт ра присвоил кредитный рейтинг пао фоса...      AAA   
4           4   5  марта  г ведущий рейтинговый аналитик юрова ал...      BBB   

  rating                                    text_lemmatized   mlrd    mln  \
0      A  повышение кредитный рейтинг акционерный общест...   True  False   
1     BB  эксперт ра подтверждать кредитный рейтинг комп...  False   True   
2      A  нкр повышать кредитный рейтинг ооо отэкопортсе...  False   True   
3    AAA  эксперт ра присваивать кредитный рейтинг пао ф...   True   True   
4    BBB  марта г ведущий рейтинговый аналитик юров алла...   True  False   

   rating_assessment  regulatory_requirements  ...

In [73]:
data.columns

Index(['Unnamed: 0', 'Id', 'text', 'category', 'rating', 'text_lemmatized',
       'mlrd', 'mln', 'rating_assessment', 'regulatory_requirements',
       'disclosure', 'credit_rating', 'joint_stock_company', 'bond_issue',
       'RUAQ', 'national_scale', 'non_financial_company', 'agency',
       'rating_activity', 'financial_instrument', 'AKR_credit_rating',
       'credit_rating_forecast', 'press_release', 'open_source_information',
       'AKR_database', 'reporting', 'additional_services',
       'conflict_of_interest', 'connection_category', 'level',
       'senior_unsecured_debt', 'emission'],
      dtype='object')

In [62]:
from sklearn.compose import ColumnTransformer

nltk.download('stopwords')
stopwords = list(nltk_stopwords.words('russian'))


y = data['rating'].values
X = data[['text_lemmatized', 'rating', 'text_lemmatized',
       'mlrd', 'mln', 'rating_assessment', 'regulatory_requirements',
       'disclosure', 'credit_rating', 'joint_stock_company', 'bond_issue',
       'RUAQ', 'national_scale', 'non_financial_company', 'agency',
       'rating_activity', 'financial_instrument', 'AKR_credit_rating',
       'credit_rating_forecast', 'press_release', 'open_source_information',
       'AKR_database', 'reporting', 'additional_services',
       'conflict_of_interest', 'connection_category', 'level',
       'senior_unsecured_debt', 'emission']]  

X_train, X_vr, y_train, y_vr = train_test_split(X, y, test_size = 0.4, random_state = 42, stratify = y)
X_valid, X_test, y_valid, y_test = train_test_split(X_vr, y_vr, test_size=0.5, random_state = 42, stratify = y_vr)


tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords)

preprocessor = ColumnTransformer(
    transformers=[
        ('tfidf', tfidf_vectorizer, 'text_lemmatized'),
    ],
    remainder='passthrough'  
)

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(random_state=1, solver='liblinear', max_iter=200))
])


param_grid = [
    {
        'model': [LGBMClassifier(random_state=42)],
        'model__num_leaves': [31, 60],
        'model__max_depth': [-1, 30],
        'model__learning_rate': [0.1, 0.01],
        'model__n_estimators': [100, 200],
        'preprocessor__tfidf__max_features': [5000, None],  
        'preprocessor__tfidf__ngram_range': [(1, 1), (1, 2)],  
    }
]


grid = GridSearchCV(pipe, param_grid=param_grid, scoring=make_scorer(f1_score, average='weighted'), cv=3, verbose=True, n_jobs=-1)


best_grid = grid.fit(X_train, y_train)


print('Best parameters are:', grid.best_params_)
print('Best score is:', grid.best_score_)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Денис\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Fitting 3 folds for each of 64 candidates, totalling 192 fits


ValueError: 
All the 192 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
192 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Денис\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Денис\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\Users\Денис\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "c:\Users\Денис\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\memory.py", line 